In [1]:
import requests

import requests
from PIL import Image

from transformers import BertJapaneseTokenizer, ViTImageProcessor, VisionEncoderDecoderModel

/home/fujie/.conda/envs/py39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = VisionEncoderDecoderModel.from_pretrained("./models/vit-bert-japanese-image-captioning_stair-captions/checkpoint-206500/")

In [3]:
tokenizer = BertJapaneseTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking")

In [4]:
image_processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")


In [6]:
# let's perform inference on an image
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)
pixel_values = image_processor(image, return_tensors="pt").pixel_values

# autoregressively generate caption (uses greedy decoding by default)
generated_ids = model.generate(pixel_values,
                               max_new_tokens=20, 
                               num_beams=5,
                               early_stopping=True,
                               do_sample=True,
                               temperature=1.2,
                                top_k=50,
                                top_p=0.95, 
                                no_repeat_ngram_size=3,
                                num_return_sequences=5)
generated_texts = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
for text in generated_texts:
    print(text)


ピンク の シーツ の 上 で 猫 と 犬 が 寝転がっ て いる
ピンク の 布 の 上 で 猫 が 二 匹 寝転がっ て いる
ピンク の 布 の 上 で 猫 が 二 匹 寝転がっ て いる
ピンク の クッション の 上 で 二 匹 の 猫 が 寝 て いる
ピンク の シーツ の 上 で 二 匹 の 猫 が 寝 て いる


In [7]:
model.save_pretrained("./models/vit-bert-japanese-image-captioning_stair-captions-results/")